# Přednáška 12: Bilineární model a bayesovská maticová dekompozice - 2. část

## Obsah přednášky:

- bayesovská formulace maticové dekompozice
- nezápornost
- model s odhadem počtu zdrojů
- aplikace

## Připomenutí problému maticové dekompozice

Mějme danou matici $D\in \mathbf{R}^{p\times n}$, která pro nás představuje data. V maticové formulaci se budeme zabývat bilineárním modelem
$$
D=AX^T,
$$
kde $A\in \mathbf{R}^{p\times r}$ a $X\in \mathbf{R}^{n\times r}$ jsou hledané matice rozkladu a $(.)^T$ značí transpozici matice. Parametr $r$ je vnitřní rozměr dekompozice.

## Nezáporná bayesovská maticová dekompozice

Na problém nezápornosti jsme narazili už u lineárních modelů, kde jsme ho řešeli volbou příslušných apriorních rozdělení jako ořezaného normálního rozdělení. Stejný princip můžeme využít i zde, ukážeme si ho nejprve na jednom rozměru. Pro jeden rozměr je ořezané normální rozdělení definováno jako
$$
\mathcal{tN}_x(\mu,\sigma,\left[a,b\right])=\frac{\sqrt{2}\exp\left(-\frac{1}{2\sigma}(x-\mu)^{2}\right)}{\sqrt{\pi\sigma}(erf(\frac{b-\mu}{\sqrt{2\sigma}})-erf(\frac{a-\mu}{\sqrt{2\sigma}}))}\chi_{\left[a,b\right]}(x).
$$
Ukažme si jeho tvar na příkladu dvou rozdělení: (i) $\mathcal{N}(1,1)$ a (ii) $t\mathcal{N}(1,1,[0,+\infty])$

<img src="img_ot/l12_tN_example.png">

Stejným způsobem můžeme modifikovat apriorní rozdělení parametrů $A$ a $X$, tedy:
$$
f\left(A\right) =\mathcal{N}\left(\mathbf{0},I_{p}\otimes I_{r},[0,+\infty]\right)\\
f\left(X\right) =\mathcal{N}\left(\mathbf{0},I_{n}\otimes I_{r},[0,+\infty]\right)
$$
které se, jak víme, replikuje i do rozdělení aposteriorního. Velkou výhodou je, že výpočet je zcela stejný jako výpočet bez pozitivity, jediný rozdíl je v netriviálním výpočtu momentů, kde 
$$
\widehat{x} =\mu-\sqrt{\sigma}\frac{\sqrt{2}[\exp(-\beta^{2})-\exp(-\alpha^{2})]}{\sqrt{\pi}(\text{erf}(\beta)-\text{erf}(\alpha))},\\
\widehat{x^{2}} =\sigma+\mu\widehat{x}-\sqrt{\sigma}\frac{\sqrt{2}[b\exp(-\beta^{2})-a\exp(-\alpha^{2})]}{\sqrt{\pi}(\text{erf}(\beta)-\text{erf}(\alpha))},
$$
kde $\alpha=\frac{a-\mu}{\sqrt{2\sigma}}$, $\beta=\frac{b-\mu}{\sqrt{2\sigma}}$ a error funkce je definovaná jako $\text{erf}(t)=\frac{2}{\sqrt{\pi}}\int_{0}^{t}e^{-u^{2}}\text{d} u$.

Toto platí pro jednorozměrné rozdělení, u vícerozměrných rozdělení je situace ještě o trochu komplikovanější a je nutné přistoupit k aproximacím, nicméně pro pochopení principu nám tato ukázka stačí. 

Podívejme se nyní na výsledky algoritmu nezáporné bayesovské maticové dekompozice na našem fantomu:
- pro $r = 2$:

<img src="img_ot/l12_vbpoz_r2.png">

- pro $r = 3$:

<img src="img_ot/l12_vbpoz_r3.png">

## Nezáporná bayesovská maticová dekompozice s odhadem počtu zdrojů

Budeme vycházet z předchozího modelu základní maticové dekompozice, který však vhodně rozšíříme, schématicky viz

<img src="img_ot/l12_graficky_DAXard.png">

Díky pricipu modularitu a nám známé komunikace mezi jednotlivými uzly pomocí momentů se můžeme věnovat modifikaci pouze uzlu $A$ a jeho hyper-parametrizování, výpočet apostreorních rozdělení pro $X$ a $\omega$ tím nijak neovlivníme. Připomeňme tedy základní model parametru $A$ jako

$$
f\left(A\right)=\mathcal{N}\left(\mathbf{0},I_{p}\otimes I_{r}\right).
$$

Tento model nyní rozšíříme pomocí ARD principu (tím jsme modelovali řídkost v lineární regresi), který aplikujeme na varianci jednotlivých zdrojů. Tím defacto říkáme, že budeme odhadovat váhu zdrojů v průběhu výpočtu. Zavedeme tedy model

$$
f\left(A\right)=\mathcal{N}\left(\mathbf{0},I_{p}\otimes V^{-1}\right),\ V=\text{diag}(\mathbf{v}),\ \mathbf{v}=[v_{1},\dots,v_{r}],
$$

a tento model doplníme o apriorní rozdělení pro vektor $\mathbf{v}$, které volíme v souladu s ARD principem jako Gamma pro každý prvek, tedy

$$
f(v_{k})=\mathcal{G}(\alpha_{0},\beta_{0}),\ \forall k.
$$

Tím máme formulovaný apriorní model (spolu s $f(D,X,\omega)$) a můžeme aplikovat variační Bayesovu metodu, zde tedy pouze na $A$, ostatní už jsme viděli.

Upravený logaritmus sdružené věrohodnosti si můžeme vyjádřit jako
$$
\ln f(D,\omega,A,X,\mathbf{v}) \propto \text{cleny od } D, X, \omega + \\
+\frac{p}{2}\sum_{k=1}^{r}\ln v_k - \frac{1}{2}\text{tr}(AVA^T) + \sum_{k=1}^{r}(\alpha_0-1)\ln v_k - \sum_{k=1}^{r}\beta_0 v_k
$$
a z něj si vyjádříme marginály pro $A$ a $\mathbf{v}$ jako
$$
\tilde{f}(A|D) \propto \exp\left( -\frac{1}{2}\widehat{\omega}\text{tr}(-2D\widehat{X}A^T+A\widehat{X^TX}A^T)-\frac{1}{2}\text{tr}(A\widehat{V}A^T) \right)\\
\tilde{f}(v_k|D) \propto \exp\left(( \alpha_{0}+\frac{p}{2} - 1 )\ln v_k - v_k( \beta_{0}+\frac{1}{2}(\widehat{A^{T}A})_{k,k} \right)
$$

Tvar nových marginál identifikujeme jako
$$
\tilde{f}(A|D)=\mathcal{N}_{A}(\mu_{A},I_{p}\otimes\Sigma_{A}),\\
\tilde{f}(v_k|D)=\mathcal{G}_{v_k}(\alpha_k,\beta_k),
$$
kde tvarovací parametry $\mu_{A},\Sigma_{A},\alpha_k,\beta_k$ dostáváme jako:
$$
\Sigma_{A}=\left(\widehat{\omega}\widehat{X^{T}X}+\widehat{V}\right)^{-1},\\
\mu_{A}=\widehat{\omega}D\widehat{X}\Sigma_{A},\\
\alpha_{k}=\alpha_{0}+\frac{p}{2},\\
\beta_{k}=\beta_{0}+\frac{1}{2}\left(\widehat{A^{T}A}\right)_{k,k}.
$$

Aplikujme opět algoritmus na naše cvičná data. Postup může být takový, že nadhodnotíme předpokládaný počet faktorů a necháme algoritmus vybrat ty relevantní (s vyššími váhami). Zobrazíme si nejen výsledný odhad $A$ a $X$, ale i vývoj vah v průběhu iterací.

<img src="img_ot/l12_vbard_r3.png">

<img src="img_ot/l12_vbard_r3_hv.png">


## Další možnosti

Jak jsme viděli již u lineární regrese, nic nám nebrání v konstrukci a kombinování složitějších modelů zahranujících např.

- předpoklad řídkosti na obrázky a/nebo na křivky
- realističtější model šumu
- fyzikální model křivek
- korelace mezi např. sousedními pixely

## Příklad aplikace - nukleární medicína

<img src="img_ot/l12_schema_scinti.png" width="800">

<img src="img_ot/l12_rnu11_data.png" width="800">

<img src="img_ot/l12_rnu11_res_r4.png" width="800">

<img src="img_ot/l12_sekv_11.jpeg" width="800">

<img src="img_ot/l12_sekv_20.jpeg" width="800">

<img src="img_ot/l12_sekv_26.jpeg" width="800">